In [4]:
! pwd

/home/jovyan/work/uwork/flight


In [14]:
! mkdir ../out_data

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [6]:
from pyspark import SQLContext
sqlContext = SQLContext(sparkContext=sc)

import pyspark.sql.functions as F

In [37]:
from pyspark.sql.window import Window

In [43]:
from pyspark.sql.types import TimestampType

In [29]:
df01 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_10_1")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))
df02 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_6_30")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))
df03 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_1_6")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))
df04 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_1_12")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))
df05 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_6_1")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))
df06 = (sqlContext.read.parquet("../s3/flight.pq.11/flight_10_14")
            .sample(False, .01, seed=0)
            .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))

# pq = df01.withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM"))
# pq2 = df02.withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM"))

In [30]:
%%time
# pq.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").save("out_data/flight.pq.11")
# pq2.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").mode("append").save("out_data/flight.pq.11")

df01.coalesce(1).write.partitionBy("fromCity", "toCity").mode("overwrite").save("../out_data/flight.pq.11_v2")
df02.coalesce(1).write.partitionBy("fromCity", "toCity").mode("append").save("../out_data/flight.pq.11_v2")
df03.coalesce(1).write.partitionBy("fromCity", "toCity").mode("append").save("../out_data/flight.pq.11_v2")
df04.coalesce(1).write.partitionBy("fromCity", "toCity").mode("append").save("../out_data/flight.pq.11_v2")
df05.coalesce(1).write.partitionBy("fromCity", "toCity").mode("append").save("../out_data/flight.pq.11_v2")
df06.coalesce(1).write.partitionBy("fromCity", "toCity").mode("append").save("../out_data/flight.pq.11_v2")

CPU times: user 236 ms, sys: 88 ms, total: 324 ms
Wall time: 30min 18s


In [31]:
df = sqlContext.read.parquet("../out_data/flight.pq.11_v2")

In [32]:
df.count()

224746

In [33]:
%%time

(df.groupby('fromCity', 'toCity')
 .agg(F.count('price').alias('count'),
      F.max('price').alias('maxPrice'))
 .orderBy('count').show())

+--------+-----------+-----+--------+
|fromCity|     toCity|count|maxPrice|
+--------+-----------+-----+--------+
| Chengdu|     Taipei|29989|  3206.1|
| Chengdu|     sydney|33502| 14297.7|
|  sydney|      Wuhan|35851|12432.62|
|  sydney|   shanghai|40458| 10943.5|
|shanghai|Los Angeles|40658| 13969.7|
|shanghai|     sydney|44288| 9578.82|
+--------+-----------+-----+--------+

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 908 ms


In [34]:
(df.groupBy('leg2_carrierSummary_airlineName')
.agg(F.count('leg2_carrierSummary_airlineName')
.alias('count')).orderBy(F.desc('count')).show(1000))

+-------------------------------+-----+
|leg2_carrierSummary_airlineName|count|
+-------------------------------+-----+
|                Xiamen Airlines|64028|
|                       AirAsiaX|48019|
|           China Southern Ai...|21031|
|                      Air China|15063|
|           China Eastern Air...|12181|
|                 Cathay Pacific| 9206|
|             Singapore Airlines| 7148|
|                               | 6768|
|               Sichuan Airlines| 5788|
|                         United| 5559|
|                 Qantas Airways| 4303|
|              American Airlines| 4042|
|                     Air Canada| 3050|
|                Hainan Airlines| 2836|
|                     Korean Air| 2554|
|                Asiana Airlines| 2438|
|                          Delta| 2042|
|              Air Macau Company| 1329|
|              Malaysia Airlines| 1203|
|             All Nippon Airways| 1046|
|            Philippine Airlines|  791|
|               Virgin Australia|  789|


In [26]:
df.select(F.weekofyear(F.col('depDate')), F.month(F.col('depDate')),
         F.dayofmonth(F.col('depDate')), F.dayofyear(F.col('depDate'))).distinct().show()

+-------------------+--------------+-------------------+------------------+
|weekofyear(depDate)|month(depDate)|dayofmonth(depDate)|dayofyear(depDate)|
+-------------------+--------------+-------------------+------------------+
|                 26|             6|                 29|               180|
|                 48|            11|                 30|               334|
|                 25|             6|                 20|               171|
|                 46|            11|                 18|               322|
|                  2|             1|                  8|                 8|
|                 46|            11|                 14|               318|
|                 52|            12|                 30|               364|
|                 42|            10|                 20|               293|
|                 51|            12|                 20|               354|
|                 52|            12|                 28|               362|
|           

In [27]:
(df.groupBy('leg1_arrivalTime_hour')
.agg(F.count('leg1_arrivalTime_hour')
.alias('count')).orderBy(F.desc('count')).show(1000))

+---------------------+-----+
|leg1_arrivalTime_hour|count|
+---------------------+-----+
|                 null|    0|
+---------------------+-----+



In [28]:
(df.groupBy('leg2_carrierSummary_displayUrgencyMessage','leg2_carrierSummary_noOfTicketsLeft')
.agg(F.count('leg2_carrierSummary_noOfTicketsLeft')
.alias('count')).orderBy('leg2_carrierSummary_displayUrgencyMessage','leg2_carrierSummary_noOfTicketsLeft').show(1000))

+-----------------------------------------+-----------------------------------+-----+
|leg2_carrierSummary_displayUrgencyMessage|leg2_carrierSummary_noOfTicketsLeft|count|
+-----------------------------------------+-----------------------------------+-----+
|                                    false|                                  0|24756|
|                                    false|                                  5|12532|
|                                    false|                                  6|  858|
|                                    false|                                  7|  960|
|                                    false|                                  8|  986|
|                                    false|                                  9|18554|
|                                     true|                                  1| 2068|
|                                     true|                                  2| 1456|
|                                     true|           

In [40]:
lookforwardDays = 19
ndays = lambda i: i * 86400 # number of seconds in a day
byVar = ['toCity', 'depDate', 'stayDays', 'timeline1_departureTime_time', 'timeline2_departureTime_time',
         'timeline1_carrier_airlineCode', 'timeline2_carrier_airlineCode']

w = (Window.partitionBy('routeCombKey')
     .orderBy(F.col('searchDate').cast('timestamp').cast('long'))
     .rowsBetween(0, ndays(lookforwardDays)))

threshold = 20

udfGetWeekNumer = lambda dt: F.udf(dt.isocalendar()[1], IntegerType())
# udfGetAvg = lambda x: F.udf(sum(x)/float(len(x)))\n\n

df2 = (df.filter(df.price > 0)
       .withColumn('routeCombKey', F.concat_ws('-', F.col('fromCity'),
                                   F.col('toCity'),
                                   F.col('stayDays'), F.col('depDate'),
                                   F.col('timeline1_departureTime_time'),
                                   F.col('timeline2_departureTime_time'),
                                   F.col('timeline1_carrier_airlineCode'),
                                   F.col('timeline2_carrier_airlineCode')))
                                   # join_udf(F.col('timeline1_departureTime_time')),
                                   # join_udf(F.col('timeline2_departureTime_time')),
                                   # join_udf(F.col('timeline1_carrier_airlineCode')),
                                   # join_udf(F.col('timeline2_carrier_airlineCode'))))
       .withColumn('futureMinPrice', F.min(F.col('price')).over(w))
       .withColumn('priceWillDrop', (F.col('price') - F.col('futureMinPrice') > threshold).cast('int'))
       .withColumn('saving0', F.col('price') - F.col('futureMinPrice'))
       .withColumn('temp0', F.lit(0))
       .withColumn('saving', F.greatest('saving0', 'temp0'))
       .drop('saving0', 'temp0')
       .withColumn('leadTime', F.datediff(F.col('depDate'), F.col('searchDate')))
       .withColumn('depWeekOfYear', F.weekofyear(F.col('depDate')))
       .withColumn('retWeekOfYear', F.weekofyear(F.col('leg2_departureTime_date')))
       .withColumn('depMonth', F.month(F.col('depDate')))
       .withColumn('depDayOfMonth', F.dayofmonth(F.col('depDate')))
       .withColumn('depDayOfYear', F.dayofyear(F.col('depDate')))
       .withColumn('depDayOfWeek', F.date_format(F.col('depDate'), 'EEEE'))
       .withColumn('retDayOfWeek', F.date_format(F.col('leg2_departureTime_date'), 'EEEE'))
       .withColumn('searchDayOfWeek', F.date_format(F.col('searchDate'), 'EEEE'))
       .withColumn('leg1_noOfTicketsLeft', 
                   F.when(F.col('leg1_carrierSummary_noOfTicketsLeft') > 0, 
                          F.col('leg1_carrierSummary_noOfTicketsLeft')).otherwise(99))
        .withColumn('leg2_noOfTicketsLeft', 
                    F.when(F.col('leg2_carrierSummary_noOfTicketsLeft') > 0, 
                           F.col('leg2_carrierSummary_noOfTicketsLeft')).otherwise(99))
       .withColumn('leg1_cabinClass_0', F.col('timeline1_carrier_cabinClass').getItem(0))
       .withColumn('leg1_cabinClass_1', F.col('timeline1_carrier_cabinClass').getItem(1))
       .withColumn('leg1_cabinClass_2', F.col('timeline1_carrier_cabinClass').getItem(2))
       .withColumn('leg2_cabinClass_0', F.col('timeline2_carrier_cabinClass').getItem(0))
       .withColumn('leg2_cabinClass_1', F.col('timeline2_carrier_cabinClass').getItem(1))
       .withColumn('leg2_cabinClass_2', F.col('timeline2_carrier_cabinClass').getItem(2))
       .select('price', 'priceWillDrop', 'futureMinPrice', 'saving',
               'fromCity', 'toCity',
               'searchDate',
               'routeCombKey',
               'leadTime', 
               'leg1_stops', 'leg2_stops',
               'leg1_noOfTicketsLeft', 'leg2_noOfTicketsLeft',
               'leg1_carrierSummary_airlineName', 'leg2_carrierSummary_airlineName',
               'leg1_departureTime_hour', 'leg2_departureTime_hour',
               'depWeekOfYear', 'depDayOfWeek','retWeekOfYear', 'retDayOfWeek',
               'searchDayOfWeek',
               'leg1_cabinClass_0', 'leg1_cabinClass_1', 'leg1_cabinClass_2',
               'leg2_cabinClass_0', 'leg2_cabinClass_1', 'leg2_cabinClass_2',
               'trip')
      )

# df2.cache()

In [41]:
df2.limit(5).toPandas()

,price,priceWillDrop,futureMinPrice,saving,fromCity,toCity,searchDate,routeCombKey,leadTime,leg1_stops,...,retWeekOfYear,retDayOfWeek,searchDayOfWeek,leg1_cabinClass_0,leg1_cabinClass_1,leg1_cabinClass_2,leg2_cabinClass_0,leg2_cabinClass_1,leg2_cabinClass_2,trip
0,478.2,0,478.2,0.0,Chengdu,Taipei,2017-06-03,Chengdu-Taipei-0-2017-06-12-10:55am-6:05pm-1:2...,9,1,...,None,None,Saturday,3,3,None,3,3,None,1
1,1332.3,0,1332.3,0.0,Chengdu,Taipei,2017-06-07,Chengdu-Taipei-0-2017-07-04-4:05pm-8:20pm-1:25...,27,1,...,None,None,Wednesday,3,2,None,3,3,None,1
2,885.4,0,885.4,0.0,Chengdu,Taipei,2017-05-25,Chengdu-Taipei-0-2017-07-04-8:20am-1:40pm-1:25...,40,1,...,None,None,Thursday,3,3,None,3,3,None,1
3,882.7,0,882.7,0.0,Chengdu,Taipei,2017-05-16,Chengdu-Taipei-0-2017-07-12-3:10pm-10:35pm-1:2...,57,1,...,None,None,Tuesday,3,3,None,3,3,None,1
4,473.1,0,473.1,0.0,Chengdu,Taipei,2017-06-06,Chengdu-Taipei-0-2017-07-29-8:35pm-8:15am-1:25...,53,1,...,None,None,Tuesday,3,3,None,3,3,None,1


In [45]:
df3 = df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
df3 = df3.withColumn('randVar', F.round(F.rand()*10, 0))
# df3.filter(F.col('searchDate') >= F.lit('2017-06-16').cast(TimestampType())).toPandas().to_pickle("D:\flight.pq.11\df3_test.pkl")
df3.groupby('randVar').agg(F.count('randVar').alias('count')).show(100)

listRand = [x['randVar'] for x in df3.select('randVar').distinct().collect()]

dfArray = [df3.where(F.col('randVar') == x) for x in listRand]

# spark.conf.set("spark.driver.maxResultSize", "10g")

i = 0

for df_temp in dfArray:
    df_temp.toPandas().to_pickle('../out_data/df3_test_' + str(listRand[i]) + '.pkl')
    i += 1

# (df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
#     .toPandas().to_pickle("../df2_v2.pkl"))
#     .limit(4000000)

# (df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
#     .toPandas().to_pickle("../df2_v2.pkl"))
#     .limit(4000000)

# get_ipython().run_cell_magic(u'time', u'', u'# pyspark --driver-memory 5g --executor-memory 25g\n\nimport pyspark.sql.functions as F\n\n# set default parquet partition to be 128mb\n# spark._jsc.hadoopConfiguration().set("dfs.block.size", "64m")\n# spark._jsc.hadoopConfiguration().setInt("parquet.block.size", 128*1024*1024)\n\n# Read in original parquet files\nparquetFile = spark.read.parquet("C:\\Data\Flight Data\\flight.pq.11\\flight_10_1")\nparquetFile2 = spark.read.parquet("C:\\Data\\Flight Data\\flight.pq.11\\flight_10_14")\n\n# Add searchMonth\nparquetFile = parquetFile.withColumn(\'searchMonth\', F.date_format(F.last_day(F.col(\'searchDate\')), "YYYYMM"))\nparquetFile2 = parquetFile2.withColumn(\'searchMonth\', F.date_format(F.last_day(F.col(\'searchDate\')), "YYYYMM"))\n\n# Save first parquet file, partition by fromCity, toCity and searchMonth\nparquetFile.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").save("C:\\Data\\Flight Data\\flight.pq.11\\flight.pq")\n\n# Save additional parquet files, partition by fromCity, toCity and searchMonth\nparquetFile2.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").mode("append").save("C:\Data\\Flight Data\\flight.pq.11\\flight.pq")\n\n# Next: convert this to a jar file, save on s3, launch ec2 spot instance and run the jar file')
# df2_in = pd.read_pickle("D:\flight.pq.11\df2.pkl")
# df2_in.searchDate.value_counts()
# parquetFile.count()
# parquetFile.show(1)
# parquetFile.select('searchDate').distinct().orderBy('searchDate').show(10000, truncate=False)
# parquetFile.select('price').distinct().orderBy('price').show(10000, truncate=False)
# parquetFile.filter(parquetFile.)
# parquetFile.select('depDate').distinct().orderBy('depDate').show(100, truncate=False)
# (parquetFile.searchDate == '2017-06-12')
# parquetFile.show(2)

+-------+-----+
|randVar|count|
+-------+-----+
|    8.0|18223|
|    0.0| 9011|
|    7.0|18258|
|    1.0|18413|
|    4.0|18386|
|    3.0|18143|
|    2.0|18068|
|   10.0| 9152|
|    6.0|18153|
|    5.0|18374|
|    9.0|18322|
+-------+-----+



In [ ]:
coding: utf-8
In[1]:

spark
In[7]:

import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import TimestampType
from pyspark.sql.window import Window
import sys
pd.set_option('display.max_columns', None)
In[3]:
df = spark.read.parquet("C:\Data\Flight Data\flight.pq.11\data\flight_v2.pq")
df.count()
In[4]:

get_ipython().run_cell_magic(u'time', u'', u"df.groupby('fromCity', 'toCity').agg(F.count('price').alias('count'), F.max('price').alias('maxPrice')).orderBy('count').show()")
In[5]:

get_ipython().run_cell_magic(u'time', u'', u"df.groupby('searchDate').agg(F.avg('price')).orderBy('searchDate').show(100)")
In[6]:

df.printSchema()
In[7]:

df.filter(F.col('searchMonth')=='201707').agg(F.avg('price')).show()
In[11]:

get_ipython().run_cell_magic(u'time', u'', u'df.dropDuplicates().count()')
In[53]:

get_ipython().run_cell_magic(u'time', u'', u'dfv1.dropDuplicates().count()')
In[54]:

get_ipython().run_cell_magic(u'time', u'', u'dfv2.dropDuplicates().count()')
In[102]:

df = spark.read.parquet("C:\Data\Flight Data\flight.pq.11\flight_10_1_14.pq")
df.count()
In[6]:

df.printSchema()
In[38]:
(df.groupBy('leg2_carrierSummary_airlineName')
.agg(F.count('leg2_carrierSummary_airlineName')
.alias('count')).orderBy(F.desc('count')).show(1000))
df.select(F.weekofyear(F.col('depDate')), F.month(F.col('depDate')),
F.dayofmonth(F.col('depDate')), F.dayofyear(F.col('depDate'))).distinct().show()
(df.groupBy('leg1_arrivalTime_hour')
.agg(F.count('leg1_arrivalTime_hour')
.alias('count')).orderBy(F.desc('count')).show(1000))
(df.groupBy('leg2_carrierSummary_displayUrgencyMessage','leg2_carrierSummary_noOfTicketsLeft')
.agg(F.count('leg2_carrierSummary_noOfTicketsLeft')
.alias('count')).orderBy('leg2_carrierSummary_displayUrgencyMessage','leg2_carrierSummary_noOfTicketsLeft').show(1000))

(df.groupBy('timeline1_carrier_bookingCode','timeline1_carrier_cabinClass')
.agg(F.count('leg2_carrierSummary_noOfTicketsLeft').alias('count'),
F.avg('price')).orderBy('timeline1_carrier_bookingCode','timeline1_carrier_cabinClass').show(1000))
timeline1_carrier_bookingCode
timeline1_carrier_cabinClass
In[4]:

df.printSchema()
In[20]:

df.limit(2).toPandas().to_csv("D:\flight.pq.11\df.csv", index=False)
In[8]:

get_ipython().run_cell_magic(u'time', u'', u"\nlookforwardDays = 19\ndays = lambda i: i * 86400 # number of seconds in a day\n\nbyVar = ['toCity', 'depDate', 'stayDays', \n 'timeline1_departureTime_time', 'timeline2_departureTime_time',\n 'timeline1_carrier_airlineCode', 'timeline2_carrier_airlineCode']\nw = (Window.partitionBy('routeCombKey')\n .orderBy(F.col('searchDate').cast('timestamp').cast('long'))\n .rowsBetween(0, days(lookforwardDays)))\nthreshold = 20\n\nudfGetWeekNumer = lambda dt: F.udf(dt.isocalendar()[1], IntegerType())\n\n# udfGetAvg = lambda x: F.udf(sum(x)/float(len(x)))\n\ndf2 = (df.filter(df.price > 0)\n .withColumn('routeCombKey', F.concat_ws('-', F.col('fromCity'), \n F.col('toCity'),\n F.col('stayDays'), F.col('depDate'), \n F.col('timeline1_departureTime_time'),\n F.col('timeline2_departureTime_time'),\n F.col('timeline1_carrier_airlineCode'),\n F.col('timeline2_carrier_airlineCode')))\n \n# join_udf(F.col('timeline1_departureTime_time')),\n# join_udf(F.col('timeline2_departureTime_time')),\n# join_udf(F.col('timeline1_carrier_airlineCode')),\n# join_udf(F.col('timeline2_carrier_airlineCode'))))\n .withColumn('futureMinPrice', F.min(F.col('price')).over(w))\n .withColumn('priceWillDrop', (F.col('price') - F.col('futureMinPrice') > threshold).cast('int'))\n .withColumn('saving0', F.col('price') - F.col('futureMinPrice'))\n .withColumn('temp0', F.lit(0))\n .withColumn('saving', F.greatest('saving0', 'temp0'))\n .drop('saving0', 'temp0')\n .withColumn('leadTime', F.datediff(F.col('depDate'), F.col('searchDate'))) \n .withColumn('depWeekOfYear', F.weekofyear(F.col('depDate')))\n .withColumn('retWeekOfYear', F.weekofyear(F.col('leg2_departureTime_date')))\n .withColumn('depMonth', F.month(F.col('depDate')))\n .withColumn('depDayOfMonth', F.dayofmonth(F.col('depDate')))\n .withColumn('depDayOfYear', F.dayofyear(F.col('depDate')))\n .withColumn('depDayOfWeek', F.date_format(F.col('depDate'), 'EEEE'))\n .withColumn('retDayOfWeek', F.date_format(F.col('leg2_departureTime_date'), 'EEEE'))\n .withColumn('searchDayOfWeek', F.date_format(F.col('searchDate'), 'EEEE'))\n .withColumn('leg1_noOfTicketsLeft', \n F.when(F.col('leg1_carrierSummary_noOfTicketsLeft') > 0, \n F.col('leg1_carrierSummary_noOfTicketsLeft')).otherwise(99)) \n .withColumn('leg2_noOfTicketsLeft', \n F.when(F.col('leg2_carrierSummary_noOfTicketsLeft') > 0, \n F.col('leg2_carrierSummary_noOfTicketsLeft')).otherwise(99))\n .withColumn('leg1_cabinClass_0', F.col('timeline1_carrier_cabinClass').getItem(0))\n .withColumn('leg1_cabinClass_1', F.col('timeline1_carrier_cabinClass').getItem(1))\n .withColumn('leg1_cabinClass_2', F.col('timeline1_carrier_cabinClass').getItem(2))\n .withColumn('leg2_cabinClass_0', F.col('timeline2_carrier_cabinClass').getItem(0))\n .withColumn('leg2_cabinClass_1', F.col('timeline2_carrier_cabinClass').getItem(1))\n .withColumn('leg2_cabinClass_2', F.col('timeline2_carrier_cabinClass').getItem(2)) \n .select('price', 'priceWillDrop', 'futureMinPrice', 'saving',\n 'fromCity', 'toCity',\n 'searchDate',\n 'routeCombKey',\n 'leadTime', \n 'leg1_stops', 'leg2_stops',\n 'leg1_noOfTicketsLeft', 'leg2_noOfTicketsLeft',\n 'leg1_carrierSummary_airlineName', 'leg2_carrierSummary_airlineName',\n 'leg1_departureTime_hour', 'leg2_departureTime_hour',\n 'depWeekOfYear', 'depDayOfWeek','retWeekOfYear', 'retDayOfWeek',\n 'searchDayOfWeek',\n 'leg1_cabinClass_0', 'leg1_cabinClass_1', 'leg1_cabinClass_2',\n 'leg2_cabinClass_0', 'leg2_cabinClass_1', 'leg2_cabinClass_2',\n 'trip')\n )\n\n, \n# df2.cache()")
In[9]:

get_ipython().run_cell_magic(u'time', u'', u'df2.limit(5).toPandas()')
In[10]:

df2.cache()
In[6]:

df3 = df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
df3 = df3.withColumn('randVar', F.round(F.rand()*10, 0))
df3.filter(F.col('searchDate') >= F.lit('2017-06-16').cast(TimestampType())).toPandas().to_pickle("D:\flight.pq.11\df3_test.pkl")
In[45]:

df3.groupby('randVar').agg(F.count('randVar').alias('count')).show(100)
In[7]:

listRand = [x['randVar'] for x in df3.select('randVar').distinct().collect()]
In[9]:

dfArray = [df3.where(F.col('randVar') == x) for x in listRand]
In[15]:

spark.conf.set("spark.driver.maxResultSize", "10g")
In[11]:

i = 0
for df_temp in dfArray:
df_temp.toPandas().to_pickle('D:\flight.pq.11\df3_test_' + str(listRand[i]) + '.pkl')
i += 1
In[16]:

(df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
.toPandas().to_pickle("D:\flight.pq.11\df2_v2.pkl"))
.limit(4000000)
In[15]:

(df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
.toPandas().to_pickle("D:\flight.pq.11\df2_v2.pkl"))
.limit(4000000)
In[18]:

get_ipython().run_cell_magic(u'time', u'', u'# pyspark --driver-memory 5g --executor-memory 25g\n\nimport pyspark.sql.functions as F\n\n# set default parquet partition to be 128mb\n# spark._jsc.hadoopConfiguration().set("dfs.block.size", "64m")\n# spark._jsc.hadoopConfiguration().setInt("parquet.block.size", 128*1024*1024)\n\n# Read in original parquet files\nparquetFile = spark.read.parquet("C:\\Data\Flight Data\\flight.pq.11\\flight_10_1")\nparquetFile2 = spark.read.parquet("C:\\Data\\Flight Data\\flight.pq.11\\flight_10_14")\n\n# Add searchMonth\nparquetFile = parquetFile.withColumn(\'searchMonth\', F.date_format(F.last_day(F.col(\'searchDate\')), "YYYYMM"))\nparquetFile2 = parquetFile2.withColumn(\'searchMonth\', F.date_format(F.last_day(F.col(\'searchDate\')), "YYYYMM"))\n\n# Save first parquet file, partition by fromCity, toCity and searchMonth\nparquetFile.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").save("C:\\Data\\Flight Data\\flight.pq.11\\flight.pq")\n\n# Save additional parquet files, partition by fromCity, toCity and searchMonth\nparquetFile2.coalesce(1).write.partitionBy("fromCity", "toCity", "searchMonth").mode("append").save("C:\Data\\Flight Data\\flight.pq.11\\flight.pq")\n\n# Next: convert this to a jar file, save on s3, launch ec2 spot instance and run the jar file')
In[8]:

df2_in = pd.read_pickle("D:\flight.pq.11\df2.pkl")
In[12]:

df2_in.searchDate.value_counts()
In[59]:
parquetFile.count()
parquetFile.show(1)
parquetFile.select('searchDate').distinct().orderBy('searchDate').show(10000, truncate=False)
parquetFile.select('price').distinct().orderBy('price').show(10000, truncate=False)
parquetFile.filter(parquetFile.)
parquetFile.select('depDate').distinct().orderBy('depDate').show(100, truncate=False)
(parquetFile.searchDate == '2017-06-12')
In[ ]:

parquetFile.show(2)